In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import bson
import pandas as pd
import datetime
import time
import json
import re
import os
import ast
import plotly.io as pio
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.colors as pc
import colorlover as cl
import json
import networkx as nx
import dash
import os
from pandas_datareader import data
from pymongo import MongoClient
from collections import Counter
from plotly.subplots import make_subplots
from datetime import datetime as dt
from bson import json_util
from dash import dcc, html
from dash.dependencies import Input, Output
import nltk
from nltk.tokenize import word_tokenize


CONNECTION TO GITHUB DATASET

In [ ]:
client = MongoClient('obfuscated')
db = client['obfuscated']
collection = db['obfuscated']
data = []
documents = collection.find()

EXTRACT ALL APIS FROM GITHUB DATASET 

In [19]:
pipeline = [


    {
    "$match": {
        "$or": [
            { "api.paths": { "$type": "object" } },
            { "api.paths": { "$type": "array" } }
        ]
    }
    },
    
    {
        '$addFields': {
           
            "auth_values_30": {
                "$map": {
                    "input": { "$objectToArray": "$api.components.securitySchemes" },
                    "as": "auth",
                    "in": "$$auth.v.type"
                }
            },
            "auth_values_20": {
                "$map": {
                    "input": { "$objectToArray": "$api.securityDefinitions" },
                    "as": "auth",
                    "in": "$$auth.v.type"
                }
            },
            "Def_in_20": {
                "$map": {
                    "input": { "$objectToArray": "$api.securityDefinitions" },
                    "as": "auth",
                    "in": "$$auth.v.in"
                }
            },

            "Def_in_30": {
                "$map": {
                    "input": { "$objectToArray": "$api.components.securitySchemes" },
                    "as": "auth",
                    "in": "$$auth.v.in"
                }
            },

             "Definitions_name_20": {
                "$map": {
                    "input": { "$objectToArray": "$api.securityDefinitions" },
                    "as": "auth",
                    "in": "$$auth.v.name"
                }
            },

            "Definitions_name_30": {
                "$map": {
                    "input": { "$objectToArray": "$api.components.securitySchemes" },
                    "as": "auth",
                    "in": "$$auth.v.name"
                }
            },

            "Desc_sec_20": {
                "$map": {
                    "input": { "$objectToArray": "$api.securityDefinitions" },
                    "as": "auth",
                    "in": "$$auth.v.description"
                }
            },

            "Desc_sec_30": {
                "$map": {
                    "input": { "$objectToArray": "$api.components.securitySchemes" },
                    "as": "auth",
                    "in": "$$auth.v.description"
                }
            },


            "scheme_defined_20": {
                "$map": {
                    "input": { "$objectToArray": "$api.securityDefinitions" },
                    "as": "auth",
                    "in": "$$auth.v.scheme"
                }
            },

             "scheme_defined_30": {
                "$map": {
                    "input": { "$objectToArray": "$api.components.securitySchemes" },
                    "as": "auth",
                    "in": "$$auth.v.scheme"
                }
            },

            "scheme_def_oauth": {
                "$map": {
                    "input": { "$objectToArray": "$api.securityDefinitions" },
                    "as": "auth",
                    "in": "$$auth.v.flow"
                }
            },


            

            "flow_def_20": {
                "$map": {
                    "input": { "$objectToArray": "$api.securityDefinitions" },
                    "as": "auth",
                    "in": "$$auth.v.flow"
                }
            },

            "flow_def_30": {
                "$map": {
                    "input": { "$objectToArray": "$api.components.securitySchemes" },
                    "as": "auth",
                    "in": "$$auth.v.flow"
                }
            },


            "op_idurl_def_20": {
                "$map": {
                    "input": { "$objectToArray": "$api.securityDefinitions" },
                    "as": "auth",
                    "in": "$$auth.v.openIdConnectUrl"
                }
            },

            "op_idurl_def_30": {
                "$map": {
                    "input": { "$objectToArray": "$api.components.securitySchemes" },
                    "as": "auth",
                    "in": "$$auth.v.openIdConnectUrl"
                }
            },


            "security": {
            '$map': {
                "input": { "$objectToArray": "$securityDefinitions" },
                "as": "sd",
                "in": {
                    "label": "$$sd.k",
                    "type":"$$sd.v.type"
                }
            }

            },
            'securityDefinitionArray': { '$objectToArray': '$api.securityDefinitions' },
            'securitySchemesArray': { '$objectToArray': '$api.components.securitySchemes' },

            "paths_total": {
                "$size": {
                "$filter": {
                    "input": { "$objectToArray": "$api.paths" },
                    "cond": { "$eq": [ { "$type": "$$this.v" }, "object" ] }
                }
            }
        },


        "api_root_security": {
                  "$cond": {
                      "if": {
                          "$or": [
                          { "$isArray": "$api.security" },
                          { "$eq": [{ "$type": "$api.security" }, "object"] }
                          ]
                      },
                      "then": 1,
                      "else": 0
                      }
              },
    

            
        
      "filteredPaths": {
          "$filter": {
              "input": {
                  "$cond": {
                      "if": { "$eq": [{ "$type": "$api.paths" }, "object"] },
                      "then": { "$objectToArray": "$api.paths" },
                      "else": "$api.paths"
                  }
              },
              "as": "path",
              "cond": {
                  "$anyElementTrue": {
                      "$map": {
                          "input": {
                              "$cond": {
                                  "if": { "$eq": [{ "$type": "$$path.v" }, "object"] },
                                  "then": { "$objectToArray": "$$path.v" },
                                  "else": { "$map": { "input": { "$literal": ["$$path.v"] }, "as": "item", "in": { "v": "$$item" } } }
                              }
                          },
                          "as": "operation",
                          "in": {
                              "$cond": {
                                  "if": { "$isArray": "$$operation.v.security" },
                                  "then": { "$ne": ["$$operation.v.security", []] },
                                  "else": False
                              }
                          }
                      }
                  }
              }
          }
      },



"methods": {
  "$size": {
    "$filter": {
      "input": {
        "$objectToArray": "$api.paths"
      },
      "as": "path",
      "cond": {
        "$and": [
          { "$eq": [{ "$type": "$$path.v" }, "object"] },
          { "$ne": [{ "$type": "$$path.v.get.security" }, "missing"] },
          { "$gt": [{ "$size": { "$ifNull": ["$$path.v.get.security", []] } }, 0] }
        ]
      }
    }
  }
},


"count_get": {
    "$size": {
        "$filter": {
            "input": {
                "$map": {
                    "input": { "$objectToArray": "$api.paths" },
                    "as": "path",
                    "in": {
                        "$cond": {
                            "if": { "$and": [
                                { "$eq": [{ "$type": "$$path.v.get" }, "object"] },
                                { "$ne": ["$$path.v.get", None] }
                            ]},
                            "then": "$$path.v.get",
                            "else": None
                        }
                    }
                }
            },
            "as": "method",
            "cond": { "$ne": ["$$method", None] }
        }
    }
},

 "count_get_security": {
    "$size": {
        "$filter": {
            "input": {
                "$map": {
                    "input": { "$objectToArray": "$api.paths" },
                    "as": "path",
                    "in": {
                        "$cond": {
                            "if": { "$and": [
                                { "$eq": [{ "$type": "$$path.v.get" }, "object"] },
                                { "$gt": [{ "$size": { "$ifNull": ["$$path.v.get.security", []] } }, 0] }
                            ]},
                            "then": "$$path.v.get",
                            "else": None
                        }
                    }
                }
            },
            "as": "method",
            "cond": { "$ne": ["$$method", None] }
        }
    }
},


"count_get_no_security": {
    "$subtract": [
        {
            "$size": {
                "$filter": {
                    "input": {
                        "$map": {
                            "input": { "$objectToArray": "$api.paths" },
                            "as": "path",
                            "in": {
                                "$cond": {
                                    "if": { "$and": [
                                        { "$eq": [{ "$type": "$$path.v.get" }, "object"] },
                                        { "$ne": ["$$path.v.get", None] }
                                    ]},
                                    "then": "$$path.v.get",
                                    "else": None
                                }
                            }
                        }
                    },
                    "as": "method",
                    "cond": { "$ne": ["$$method", None] }
                }
            }
        },
        {
            "$size": {
                "$filter": {
                    "input": {
                        "$map": {
                            "input": { "$objectToArray": "$api.paths" },
                            "as": "path",
                            "in": {
                                "$cond": {
                                    "if": { "$and": [
                                        { "$eq": [{ "$type": "$$path.v.get" }, "object"] },
                                        { "$gt": [{ "$size": { "$ifNull": ["$$path.v.get.security", []] } }, 0] }
                                    ]},
                                    "then": "$$path.v.get",
                                    "else": None
                                }
                            }
                        }
                    },
                    "as": "method",
                    "cond": { "$ne": ["$$method", None] }
                }
            }
        }
    ]
},






"count_post": {
    "$size": {
        "$filter": {
            "input": {
                "$map": {
                    "input": { "$objectToArray": "$api.paths" },
                    "as": "path",
                    "in": {
                        "$cond": {
                            "if": { "$and": [
                                { "$eq": [{ "$type": "$$path.v.post" }, "object"] },
                                { "$ne": ["$$path.v.post", None] }
                            ]},
                            "then": "$$path.v.post",
                            "else": None
                        }
                    }
                }
            },
            "as": "method",
            "cond": { "$ne": ["$$method", None] }
        }
    }
},

 "count_post_security": {
    "$size": {
        "$filter": {
            "input": {
                "$map": {
                    "input": { "$objectToArray": "$api.paths" },
                    "as": "path",
                    "in": {
                        "$cond": {
                            "if": { "$and": [
                                { "$eq": [{ "$type": "$$path.v.post" }, "object"] },
                                { "$gt": [{ "$size": { "$ifNull": ["$$path.v.post.security", []] } }, 0] }
                            ]},
                            "then": "$$path.v.post",
                            "else": None
                        }
                    }
                }
            },
            "as": "method",
            "cond": { "$ne": ["$$method", None] }
        }
    }
},


"count_post_no_security": {
    "$subtract": [
        {
            "$size": {
                "$filter": {
                    "input": {
                        "$map": {
                            "input": { "$objectToArray": "$api.paths" },
                            "as": "path",
                            "in": {
                                "$cond": {
                                    "if": { "$and": [
                                        { "$eq": [{ "$type": "$$path.v.post" }, "object"] },
                                        { "$ne": ["$$path.v.post", None] }
                                    ]},
                                    "then": "$$path.v.post",
                                    "else": None
                                }
                            }
                        }
                    },
                    "as": "method",
                    "cond": { "$ne": ["$$method", None] }
                }
            }
        },
        {
            "$size": {
                "$filter": {
                    "input": {
                        "$map": {
                            "input": { "$objectToArray": "$api.paths" },
                            "as": "path",
                            "in": {
                                "$cond": {
                                    "if": { "$and": [
                                        { "$eq": [{ "$type": "$$path.v.post" }, "object"] },
                                        { "$gt": [{ "$size": { "$ifNull": ["$$path.v.post.security", []] } }, 0] }
                                    ]},
                                    "then": "$$path.v.post",
                                    "else": None
                                }
                            }
                        }
                    },
                    "as": "method",
                    "cond": { "$ne": ["$$method", None] }
                }
            }
        }
    ]
},




"count_put": {
    "$size": {
        "$filter": {
            "input": {
                "$map": {
                    "input": { "$objectToArray": "$api.paths" },
                    "as": "path",
                    "in": {
                        "$cond": {
                            "if": { "$and": [
                                { "$eq": [{ "$type": "$$path.v.put" }, "object"] },
                                { "$ne": ["$$path.v.put", None] }
                            ]},
                            "then": "$$path.v.put",
                            "else": None
                        }
                    }
                }
            },
            "as": "method",
            "cond": { "$ne": ["$$method", None] }
        }
    }
},

 "count_put_security": {
    "$size": {
        "$filter": {
            "input": {
                "$map": {
                    "input": { "$objectToArray": "$api.paths" },
                    "as": "path",
                    "in": {
                        "$cond": {
                            "if": { "$and": [
                                { "$eq": [{ "$type": "$$path.v.put" }, "object"] },
                                { "$gt": [{ "$size": { "$ifNull": ["$$path.v.put.security", []] } }, 0] }
                            ]},
                            "then": "$$path.v.put",
                            "else": None
                        }
                    }
                }
            },
            "as": "method",
            "cond": { "$ne": ["$$method", None] }
        }
    }
},


"count_put_no_security": {
    "$subtract": [
        {
            "$size": {
                "$filter": {
                    "input": {
                        "$map": {
                            "input": { "$objectToArray": "$api.paths" },
                            "as": "path",
                            "in": {
                                "$cond": {
                                    "if": { "$and": [
                                        { "$eq": [{ "$type": "$$path.v.put" }, "object"] },
                                        { "$ne": ["$$path.v.put", None] }
                                    ]},
                                    "then": "$$path.v.put",
                                    "else": None
                                }
                            }
                        }
                    },
                    "as": "method",
                    "cond": { "$ne": ["$$method", None] }
                }
            }
        },
        {
            "$size": {
                "$filter": {
                    "input": {
                        "$map": {
                            "input": { "$objectToArray": "$api.paths" },
                            "as": "path",
                            "in": {
                                "$cond": {
                                    "if": { "$and": [
                                        { "$eq": [{ "$type": "$$path.v.put" }, "object"] },
                                        { "$gt": [{ "$size": { "$ifNull": ["$$path.v.put.security", []] } }, 0] }
                                    ]},
                                    "then": "$$path.v.put",
                                    "else": None
                                }
                            }
                        }
                    },
                    "as": "method",
                    "cond": { "$ne": ["$$method", None] }
                }
            }
        }
    ]
},





"count_delete": {
    "$size": {
        "$filter": {
            "input": {
                "$map": {
                    "input": { "$objectToArray": "$api.paths" },
                    "as": "path",
                    "in": {
                        "$cond": {
                            "if": { "$and": [
                                { "$eq": [{ "$type": "$$path.v.delete" }, "object"] },
                                { "$ne": ["$$path.v.delete", None] }
                            ]},
                            "then": "$$path.v.delete",
                            "else": None
                        }
                    }
                }
            },
            "as": "method",
            "cond": { "$ne": ["$$method", None] }
        }
    }
},

 "count_delete_security": {
    "$size": {
        "$filter": {
            "input": {
                "$map": {
                    "input": { "$objectToArray": "$api.paths" },
                    "as": "path",
                    "in": {
                        "$cond": {
                            "if": { "$and": [
                                { "$eq": [{ "$type": "$$path.v.delete" }, "object"] },
                                { "$gt": [{ "$size": { "$ifNull": ["$$path.v.delete.security", []] } }, 0] }
                            ]},
                            "then": "$$path.v.delete",
                            "else": None
                        }
                    }
                }
            },
            "as": "method",
            "cond": { "$ne": ["$$method", None] }
        }
    }
},


"count_delete_no_security": {
    "$subtract": [
        {
            "$size": {
                "$filter": {
                    "input": {
                        "$map": {
                            "input": { "$objectToArray": "$api.paths" },
                            "as": "path",
                            "in": {
                                "$cond": {
                                    "if": { "$and": [
                                        { "$eq": [{ "$type": "$$path.v.delete" }, "object"] },
                                        { "$ne": ["$$path.v.delete", None] }
                                    ]},
                                    "then": "$$path.v.delete",
                                    "else": None
                                }
                            }
                        }
                    },
                    "as": "method",
                    "cond": { "$ne": ["$$method", None] }
                }
            }
        },
        {
            "$size": {
                "$filter": {
                    "input": {
                        "$map": {
                            "input": { "$objectToArray": "$api.paths" },
                            "as": "path",
                            "in": {
                                "$cond": {
                                    "if": { "$and": [
                                        { "$eq": [{ "$type": "$$path.v.delete" }, "object"] },
                                        { "$gt": [{ "$size": { "$ifNull": ["$$path.v.delete.security", []] } }, 0] }
                                    ]},
                                    "then": "$$path.v.delete",
                                    "else": None
                                }
                            }
                        }
                    },
                    "as": "method",
                    "cond": { "$ne": ["$$method", None] }
                }
            }
        }
    ]
},







  },
      
 },
    
    {
        '$project': {
            'commit_id': '$id',
            'apiSpec_id': '$api_spec_id',
            'timeStamp': '$commit_date',
            'paths_total':1,
            'paths_local_security': {
                "$size": "$filteredPaths"
            },            
            'paths_nolocal_security': {
            '$subtract': ['$paths_total', {
                '$size': '$filteredPaths'
            }]
            }, 

            "count_get": 1,
            "count_get_security":1,
            "count_get_no_security": 1,
             "count_post": 1,
            "count_post_security":1,
            "count_post_no_security": 1,
             "count_put": 1,
            "count_put_security":1,
            "count_put_no_security": 1,
             "count_delete": 1,
            "count_delete_security":1,
            "count_delete_no_security": 1,
            'api_root_security':1,
            'schemas_size': '$schemaSize.schemas',
            'security_size': 1,
            'auth_values_30': 1,
            'auth_values_20': 1,
            'Definitions_name_20':1,
            'Definitions_name_30':1,
            'Desc_sec_20':1,
            'Desc_sec_30':1,
            'Def_in_20':1,
            'Def_in_30':1,
            'scheme_def':1,
            'scheme_defined_20':1,
            'scheme_defined_30':1,
            'flow_def_20':1,
            'flow_def_30':1,
            'op_idurl_def_20':1,
            'op_idurl_def_30':1,
            'securityDefinitionNames': {
                '$map': {
                    'input': '$securityDefinitionArray',
                    'as': 'sd',
                    'in': '$$sd.k'
                },
            },

            'securitySchemesNames': {
                '$map': {
                    'input': '$securitySchemesArray',
                    'as': 'sd',
                    'in': '$$sd.k'
                }
            }
        }
    }
]

results = list(collection.aggregate(pipeline))
df_todoeldataset = pd.DataFrame(results)

ADDING SECURITY SCHEMES COLUMNS

In [ ]:
def count_elements(lst):
    if lst is None:
        return 0
    return len(lst)

def concatenate_lists(list1, list2):
    if list1 is None:
        list1 = []
    if list2 is None:
        list2 = []
    return [str(item) for item in list1] + [str(item) for item in list2]


def combinar_columnas(row):
    if row['count_security_definitions'] == 0:
        return row['count_security_schemes']
    elif row['count_security_schemes'] == 0:
        return row['count_security_definitions']
    else:
        return min(row['count_security_definitions'], row['count_security_schemes'])


df_todoeldataset['count_security_definitions'] = df_todoeldataset['auth_values_20'].apply(count_elements)
df_todoeldataset['count_security_schemes'] = df_todoeldataset['auth_values_30'].apply(count_elements)
df_todoeldataset['security_combined'] = df_todoeldataset.apply(lambda row: combinar_columnas(row), axis=1)
df_todoeldataset['security_names'] = df_todoeldataset.apply(lambda row: ', '.join(concatenate_lists(row['auth_values_20'], row['auth_values_30'])), axis=1)


def count_security_level(security_names):
    if security_names is None:
        return 0
    unique_components = set(filter(None, map(str.strip, security_names.split(','))))
    return len(unique_components)

df_todoeldataset['sec_level'] = df_todoeldataset['security_names'].apply(count_security_level)

def determine_auth_label(row):
    if np.any(pd.notna(row['auth_values_30'])):
        return 'API 3'
    elif np.any(pd.notna(row['auth_values_20'])):
        return 'API 2'
    else:
        return 'Not defined'

df_todoeldataset['auth_label'] = df_todoeldataset.apply(determine_auth_label, axis=1)

df_todoeldataset.to_csv('tableallapis_githubdb.csv', index=False)   

folder_path_csv = 'csv'
csv_file_path = os.path.join(folder_path_csv, 'tableallapis_githubdb.csv')
df_todoeldataset.to_csv(csv_file_path, index=False)
